In [32]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
from sklearn import preprocessing, svm
from sklearn.model_selection import cross_validate, train_test_split
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib notebook

style.use('ggplot')

In [33]:
df = pd.read_excel("titanic.xls")
o_df = pd.DataFrame.copy(df)
df.drop(['body', 'name'], 1, inplace=True)
df._convert(numeric=True)
df.fillna(0, inplace=True)

In [34]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    
    for column in columns:
        text = {}
        def convert_to_int(val):
            return text[val]
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text:
                    text[unique] = x
                    x+=1
            df[column] = list(map(convert_to_int, df[column]))   #set list map
            
    return df

In [35]:
df = handle_non_numerical_data(df)

x = np.array(df.drop(['survived'], 1).astype(float))
x = preprocessing.scale(x) #important
y = np.array(df['survived'])

clf = MeanShift()
clf.fit(x)

MeanShift()

In [36]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_

In [37]:
o_df['cluster_gp'] = np.nan

for i in range(len(x)):
    o_df['cluster_gp'].iloc[i] = labels[i]
    
    
n_clusters_ = len(np.unique(labels))
survival_rates = {}
for i in range(n_clusters_):
    temp_df = o_df[ (o_df['cluster_gp']==float(i)) ]
    survival_cluster = temp_df[ (temp_df['survived']==1) ]
    survival_rate = len(survival_cluster) / len(temp_df)
    survival_rates[i] = survival_rate
print(survival_rates)

{0: 0.3822825219473264, 1: 0.08333333333333333, 2: 0.8181818181818182, 3: 0.1}


C:\Users\s2007\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [47]:
print(o_df[ (o_df['cluster_gp']==0) ].describe())

            pclass     survived          sex          age        sibsp  \
count  1253.000000  1253.000000  1253.000000  1253.000000  1253.000000   
mean      2.298484     0.382283     0.346369    23.943469     0.386273   
std       0.830897     0.486139     0.476002    17.528302     0.707696   
min       1.000000     0.000000     0.000000     0.000000     0.000000   
25%       2.000000     0.000000     0.000000     8.000000     0.000000   
50%       3.000000     0.000000     0.000000    24.000000     0.000000   
75%       3.000000     1.000000     1.000000    35.000000     1.000000   
max       3.000000     1.000000     1.000000    80.000000     4.000000   

             parch       ticket         fare        cabin     embarked  \
count  1253.000000  1253.000000  1253.000000  1253.000000  1253.000000   
mean      0.289705   466.311253    28.163324    20.672785     2.102155   
std       0.635628   268.826997    36.241986    46.595824     0.544303   
min       0.000000     0.000000     0